In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime


# Extracting a priori stock targets for index rebalancing arbitrage

Data source: Bloomberg for index constitution 1 week prior to rank date.<br>

Targeting the FTSE 100 Index, one week prior to rank date we extract the index constitution date to identify bottom x\% of the FTSE 100 Index as the target stocks for exclusions, and top x\% of the FTSE 250 Index as the target stocks for inclusions.<br>

The reason for using a fixed percentage instead of absolute number is that since FTSE 100 is large cap and FTSE 250 is mid cap, we can expect that in the mid cap the price is more volatile, and thus market cap will be more volatile. So a fixed percentage allows us to consider a wider range of mid cap for target inclusion.

In [2]:
# date pairs
dates =\
[
    '20220524',
    '20220823',
    '20221122',
    '20230221',
    '20230523',
]


In [3]:
def get_constituent_list(index, date):
    
    # Extract dataframe for pre-review data
    df =\
    (
        pd
        .read_excel(
            f"../FTSE/{index}_pre-review.xlsx", 
            sheet_name = date
        )
        .set_index('Ticker')
    )
    
    # Filter out nonsense data
    df =\
    (
        df[df['Market Cap\n'] != '--']
        .sort_values(by = 'Market Cap\n', ascending = False)
    )
    
    return df


In [4]:
def get_target_stocks(date, pct):
    
    # Extract required data
    ftse100 = get_constituent_list('UKX', date)
    ftse100['Index'] = "FTSE100"
    ftse250 = get_constituent_list('MCX', date)
    ftse250['Index'] = "FTSE250"
    
    # Get bottom of FTSE100
    bottom_ftse100 =\
    (
        ftse100
        .tail(int(100*pct))
    )
    
    # Get top of FTSE250
    top_ftse250 =\
    (
        ftse250
        .head(int(250*pct))
    )
    
    # Combine bottoms and tops
    target_stocks =\
    (
        pd
        .concat(
            [bottom_ftse100,top_ftse250]
        )
    )
    
    target_stocks.columns =\
    [
        col.replace('\n','') for col in target_stocks.columns 
    ]
    
    return target_stocks
    
    

In [5]:
target_stock_universe =\
(
    map(
        get_target_stocks, 
        dates, 
        [0.10]*len(dates)
    )
)

# Write into excel file
with pd.ExcelWriter('../output/target_stock_universe.xlsx') as writer:  
    for df, date in zip(list(target_stock_universe), dates):
        df\
        .to_excel(
            writer, 
            sheet_name = date
        )

